This code was borrowed from AAI 540 labs

https://github.com/mechristenson/aai-540-labs.git

## Create S3 Bucket

In [1]:
#!pip install --upgrade boto3 botocore awscli

In [2]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
setup_s3_bucket_passed = False

In [4]:
print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-590183687297


Verify S3_BUCKET Bucket Creation

In [5]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': 'Z8T1K4VZRAHXFJC2', 'HostId': '0CaeCvCojLhmdqLc70YUNkDLW78nQW3QkBeeNPrlw8v+2E32AeU5TQf27MyE13+4jc1qyDCq9UU=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '0CaeCvCojLhmdqLc70YUNkDLW78nQW3QkBeeNPrlw8v+2E32AeU5TQf27MyE13+4jc1qyDCq9UU=', 'x-amz-request-id': 'Z8T1K4VZRAHXFJC2', 'date': 'Sat, 13 Sep 2025 04:24:52 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}


In [6]:
%store setup_s3_bucket_passed


Stored 'setup_s3_bucket_passed' (bool)


In [7]:
%store


Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path_parquet                    -> 's3://sagemaker-us-east-1-590183687297/toxicity_pd
setup_s3_bucket_passed                     -> True


## Set up Data lake

In [8]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

#### Set S3 Destination Folder

In [9]:
s3_private_path_parquet = "s3://{}/toxicity_pds/parquet".format(bucket)
print(s3_private_path_parquet)

s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet


In [10]:
%store s3_private_path_parquet

Stored 's3_private_path_parquet' (str)


In [11]:
%store -r setup_s3_bucket_passed

In [12]:
try:
    setup_s3_bucket_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++")


In [13]:
if not setup_s3_bucket_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

### Copy data from github to S3 bucket

In [14]:
!aws s3 cp --recursive /home/sagemaker-user/aai540_toxicity_classification/civil $s3_private_path_parquet/ --include "*"


upload: civil/test-00000-of-00001.parquet to s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet/test-00000-of-00001.parquet
upload: civil/validation-00000-of-00001.parquet to s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet/validation-00000-of-00001.parquet
upload: civil/train-00000-of-00001.parquet to s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet/train-00000-of-00001.parquet


### List the files

In [15]:
print(s3_private_path_parquet)

s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet


In [16]:
!aws s3 ls $s3_private_path_parquet/

2025-09-13 04:24:54   34099179 test-00000-of-00001.parquet
2025-09-13 04:24:54   68844404 train-00000-of-00001.parquet
2025-09-13 04:24:54   11697541 validation-00000-of-00001.parquet


In [17]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path_parquet                    -> 's3://sagemaker-us-east-1-590183687297/toxicity_pd
setup_s3_bucket_passed                     -> True


## Create Athena Database Schema

In [18]:
ingest_create_athena_db_passed = False

In [19]:
%store -r s3_private_path_parquet

In [20]:
try:
    s3_private_path_parquet
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

In [21]:
print(s3_private_path_parquet)

s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet


### Import PyAthena

In [22]:
from pyathena import connect

### Create Athena Database

In [23]:
database_name = "aai540_toxicity_aws"

In [24]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [25]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [26]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS aai540_toxicity_aws


In [27]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_4868/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


### Verify The Database Has Been Created Succesfully

In [28]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_4868/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,aai540_toxicity_aws
1,default


In [29]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [30]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


In [31]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path_parquet                    -> 's3://sagemaker-us-east-1-590183687297/toxicity_pd
setup_s3_bucket_passed                     -> True


### Create Tables

In [32]:
ingest_create_athena_table_pqt_passed = False

In [33]:
table_name = 'toxicity_pqt'
train = '%/train-00000-of-00001.parquet'
validation = '%/validation-00000-of-00001.parquet'
test = '%/test-00000-of-00001.parquet'

In [34]:
s3_private_path_parquet

's3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet'

In [35]:
drop_stmt = """DROP TABLE IF EXISTS {}.{}""".format(database_name, table_name)
pd.read_sql(drop_stmt, conn)

/tmp/ipykernel_4868/2386943986.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(drop_stmt, conn)


""


In [36]:
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
  uid               BIGINT,
  id                BIGINT,
  comment_text      STRING,
  toxicity          BIGINT,
  has_active_attrs  BOOLEAN,
  active_attrs      ARRAY<string>,
  male              BIGINT,
  female            BIGINT,
  lgbtq             BIGINT,
  christian         BIGINT,
  muslim            BIGINT,
  other_religions   BIGINT,
  black             BIGINT,
  white             BIGINT,
  identity_any      BIGINT,
  severe_toxicity   BIGINT,
  obscene           BIGINT,
  threat            BIGINT,
  insult            BIGINT,
  identity_attack   BIGINT,
  sexual_explicit   BIGINT
)
STORED AS PARQUET
LOCATION '{}/'""".format(database_name, table_name, s3_private_path_parquet)

print(statement)
pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS aai540_toxicity_aws.toxicity_pqt (
  uid               BIGINT,
  id                BIGINT,
  comment_text      STRING,
  toxicity          BIGINT,
  has_active_attrs  BOOLEAN,
  active_attrs      ARRAY<string>,
  male              BIGINT,
  female            BIGINT,
  lgbtq             BIGINT,
  christian         BIGINT,
  muslim            BIGINT,
  other_religions   BIGINT,
  black             BIGINT,
  white             BIGINT,
  identity_any      BIGINT,
  severe_toxicity   BIGINT,
  obscene           BIGINT,
  threat            BIGINT,
  insult            BIGINT,
  identity_attack   BIGINT,
  sexual_explicit   BIGINT
)
STORED AS PARQUET
LOCATION 's3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet/'


/tmp/ipykernel_4868/117541428.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [37]:
train_view = 'toxicity_train'
statement_view = """CREATE OR REPLACE VIEW {}.{} AS SELECT * FROM {}.{} WHERE "$path" LIKE '{}'""".format(
    database_name,
    train_view,
    database_name,
    table_name,
    train
)
print(statement_view)
pd.read_sql(statement_view, conn)

CREATE OR REPLACE VIEW aai540_toxicity_aws.toxicity_train AS SELECT * FROM aai540_toxicity_aws.toxicity_pqt WHERE "$path" LIKE '%/train-00000-of-00001.parquet'


/tmp/ipykernel_4868/2100174071.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_view, conn)


""


In [38]:
validation_view = 'toxicity_val'
statement_view = """CREATE OR REPLACE VIEW {}.{} AS SELECT * FROM {}.{} WHERE "$path" LIKE '{}'""".format(
    database_name,
    validation_view,
    database_name,
    table_name,
    validation
)
print(statement_view)
pd.read_sql(statement_view, conn)

CREATE OR REPLACE VIEW aai540_toxicity_aws.toxicity_val AS SELECT * FROM aai540_toxicity_aws.toxicity_pqt WHERE "$path" LIKE '%/validation-00000-of-00001.parquet'


/tmp/ipykernel_4868/3418994591.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_view, conn)


""


In [39]:
test_view = 'toxicity_test'
statement_view = """CREATE OR REPLACE VIEW {}.{} AS SELECT * FROM {}.{} WHERE "$path" LIKE '{}'""".format(
    database_name,
    test_view,
    database_name,
    table_name,
    test
)
print(statement_view)
pd.read_sql(statement_view, conn)


CREATE OR REPLACE VIEW aai540_toxicity_aws.toxicity_test AS SELECT * FROM aai540_toxicity_aws.toxicity_pqt WHERE "$path" LIKE '%/test-00000-of-00001.parquet'


/tmp/ipykernel_4868/1627434926.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_view, conn)


""


### Verify The Table Has Been Created Succesfully

In [40]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_4868/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,toxicity_pqt
1,toxicity_test
2,toxicity_train
3,toxicity_val


In [41]:
if table_name in df_show.values:
    ingest_create_athena_table_pqt_passed = True

print(ingest_create_athena_table_pqt_passed)

True


### Run A Sample SQL Query

In [42]:
statement = """SELECT * FROM {}.{}""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM aai540_toxicity_aws.toxicity_pqt


/tmp/ipykernel_4868/2419674085.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,uid,id,comment_text,toxicity,has_active_attrs,active_attrs,male,female,lgbtq,christian,...,other_religions,black,white,identity_any,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit
0,18,5711489,𝑯𝒆 𝒕𝒐𝒍𝒅 𝒑𝒐𝒍𝒊𝒄𝒆 𝒕𝒉𝒂𝒕 𝒉𝒆 𝒃𝒖𝒚𝒔 𝒄𝒐𝒄𝒂𝒊𝒏𝒆 “𝒊𝒏 𝒍𝒂𝒓𝒈𝒆 ...,0,False,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21,5295059,为加拿大和中国友谊祝贺。与其想象中国人对加拿大人的看法不如直接去问，不要用自己的想法去猜。本...,0,True,[christian],0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,26,5753025,♫ Mister we could use a man like Neville Chamb...,0,True,[male],1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,47,6100226,…so much for your opinion.,0,False,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,54,974066,…as most everyone knows both parties completel...,0,False,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
statement = "SHOW VIEWS in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_4868/3130455028.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,views
0,toxicity_test
1,toxicity_train
2,toxicity_val


In [44]:
statement = """SELECT * FROM {}.{}""".format(
    database_name, test_view
)

print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM aai540_toxicity_aws.toxicity_test


/tmp/ipykernel_4868/1810532948.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,uid,id,comment_text,toxicity,has_active_attrs,active_attrs,male,female,lgbtq,christian,...,other_religions,black,white,identity_any,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit
0,1,7099334,😳 mean,0,False,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,5848857,😥,0,False,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,7191701,😅😅😅😅😅😅😅😅😅😅. Evangelicals call on Trump😂😂😂😂😂😂.,0,True,[christian],0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
3,7,6237092,"😂😂😂 she's my lover, man that Suzie Wong 😂😂",0,True,[male],1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,12,6294419,💙 ScreenViewsHD 💙 is the best to watch nf...,0,False,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Release Resources

In [46]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:

%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}